In [1]:
from martypy import Marty
my_marty = Marty("wifi","192.168.252.234")
my_marty

sendRICRESTURLSync msgNum 2 response is not JSON
Traceback (most recent call last):
  File "/Users/arromaljj/ENTER/envs/liva/lib/python3.10/site-packages/martypy/RICInterface.py", line 255, in waitForSyncResult
    respObj = json.loads(respStr.payload.rstrip('\0'))
  File "/Users/arromaljj/ENTER/envs/liva/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/arromaljj/ENTER/envs/liva/lib/python3.10/json/decoder.py", line 340, in decode
    raise JSONDecodeError("Extra data", s, end)
json.decoder.JSONDecodeError: Extra data: line 1 column 5 (char 4)
sendRICRESTURLSync msgNum 2 response is not JSON
Traceback (most recent call last):
  File "/Users/arromaljj/ENTER/envs/liva/lib/python3.10/site-packages/martypy/RICInterface.py", line 255, in waitForSyncResult
    respObj = json.loads(respStr.payload.rstrip('\0'))
  File "/Users/arromaljj/ENTER/envs/liva/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decod

In [2]:
from dotenv import load_dotenv
load_dotenv()


True

## Actions


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
template = ChatPromptTemplate.from_messages(
    [( "system",    "You are a story telling robot called Marty. Given the following emotion, you need to tell a story that matches the emotion. Keep the story short and concise."),
    ("user", "  Please tell a story that matches the emotion: {emotion}. Start the story directly as though you are speaking to a child as Marty.")]
)   

chain = template | model





In [4]:
result = chain.invoke({"emotion": "happy"})

In [5]:
result.content

'Once upon a time, in a bright and colorful village called Sunnyvale, there lived a little girl named Lily. Lily had the most beautiful smile that could light up even the cloudiest of days! She loved to explore the fields of wildflowers that surrounded her home, where butterflies danced and birds sang sweet melodies.\n\nOne sunny morning, Lily woke up with a twinkle in her eye. Today was the day of the Great Sunnyvale Picnic, a special event that happened only once a year! Everyone in the village would gather at the big, shady oak tree in the center of the park, bringing their favorite foods and games to share.\n\nLily quickly dressed in her favorite yellow dress, which matched the sunflowers blooming in her garden. She packed a basket full of delicious treats: peanut butter and jelly sandwiches, fresh strawberries, and her mom’s famous chocolate chip cookies. With her basket in hand, she skipped down the path, humming a cheerful tune.\n\nWhen she arrived at the park, her eyes sparkled

In [6]:
str_arr = result.content.split(" ")
# Chunk str_arr into arrays of 10 words
chunk_size = 10
chunks = [str_arr[i:i+chunk_size] for i in range(0, len(str_arr), chunk_size)]
grouped = [" ".join(chunk) for chunk in chunks]
grouped


['Once upon a time, in a bright and colorful village',
 'called Sunnyvale, there lived a little girl named Lily. Lily',
 'had the most beautiful smile that could light up even',
 'the cloudiest of days! She loved to explore the fields',
 'of wildflowers that surrounded her home, where butterflies danced and',
 'birds sang sweet melodies.\n\nOne sunny morning, Lily woke up with',
 'a twinkle in her eye. Today was the day of',
 'the Great Sunnyvale Picnic, a special event that happened only',
 'once a year! Everyone in the village would gather at',
 'the big, shady oak tree in the center of the',
 'park, bringing their favorite foods and games to share.\n\nLily quickly',
 'dressed in her favorite yellow dress, which matched the sunflowers',
 'blooming in her garden. She packed a basket full of',
 'delicious treats: peanut butter and jelly sandwiches, fresh strawberries, and',
 'her mom’s famous chocolate chip cookies. With her basket in',
 'hand, she skipped down the path, humming a chee

In [7]:
import time
for chunk in grouped[0:10]:
    my_marty.speak(chunk)
    time.sleep(1)


In [19]:

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate


class Emotion(BaseModel):
    """The detected emotion of the user"""
    emotion: str = Field(
        description="The emotion to detect. Choose one of the following: happy, sad, angry, excited, bored, curious, or confused.",
    )



    
@tool
def detected_emotion(detected_emotion: Emotion):
    """The function is called to determine the emotion of the user. The argument is the detected emotion. """
    return detected_emotion




template = ChatPromptTemplate.from_messages(
    [( "system",    "You are an emotion detection agent. You are given a sentence and you need to detect the emotion of the user."),
    ("user", " User Speech: {speech}")]
)   


model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [detected_emotion]
model_with_tools = model.bind_tools(tools, tool_choice="required")

chain = template | model_with_tools


def detect_emotion(speech: str) -> Emotion:
    result = chain.invoke({"speech": speech})
    return result.tool_calls[0]['args']['detected_emotion']['emotion']

















In [20]:
## Come with some good test phrases to test the emotion detection
testCases = [
    "I am having some pain but I am liking this ice cream a lot.",
    "I am a bit tired today but I am happy to see you.",
    "I am really excited to see you.",
    "I am really sad to see you.",
    "I am really angry at you.",
]


In [21]:
for case in testCases:
    result = detect_emotion(case)
    print(f"Test Case: {case}")
    print(f"Detected Emotion: {result}")
    print("\n")


Test Case: I am having some pain but I am liking this ice cream a lot.
Detected Emotion: sad


Test Case: I am a bit tired today but I am happy to see you.
Detected Emotion: happy


Test Case: I am really excited to see you.
Detected Emotion: excited


Test Case: I am really sad to see you.
Detected Emotion: sad


Test Case: I am really angry at you.
Detected Emotion: angry




In [18]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uKXKb2BgdRp0DdInHdp3nBse', 'function': {'arguments': '{"detected_emotion": {"emotion": "sad"}}', 'name': 'detected_emotion'}, 'type': 'function'}, {'id': 'call_C7VPAkpFeXky0x79698rGeg0', 'function': {'arguments': '{"detected_emotion": {"emotion": "happy"}}', 'name': 'detected_emotion'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 139, 'total_tokens': 198, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2c044c5b-c2b4-49a4-9543-46ee61ccffbf-0', tool_calls=[{'name': 'detected_emotion', 'args': {'detected_emotion': {'emotion': 'sad'}}, 'id': 'call_uKXKb2B

## Flows


In [ ]:

def startup():
    # Get ready, say hi, wave hands, greet time,  etc
    pass


def happy_flow():
    pass

def sad_flow():
    pass



## Events

In [ ]:
def on_emotion(emotion):
    if emotion == "happy":
        my_marty.dance()
    elif emotion == "sad":
        my_marty.cry()



In [1]:
import sounddevice as sd
import numpy as np
import webrtcvad
import wave
import keyboard
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class RecorderConfig:
    """Configuration settings for the voice recorder"""
    sample_rate: int = 16000
    chunk_duration_ms: int = 30
    vad_aggressiveness: int = 3
    silence_threshold: int = 30
    stop_key: str = 'esc'

class AudioProcessor:
    """Handles audio processing operations"""
    def __init__(self, sample_rate: int):
        self.sample_rate = sample_rate
        self.vad = webrtcvad.Vad(3)
    
    def convert_to_int16(self, float32_data: np.ndarray) -> np.ndarray:
        """Convert float32 audio data to int16"""
        return (float32_data * 32767).astype(np.int16)
    
    def is_speech(self, audio_data: np.ndarray) -> bool:
        """Detect if audio chunk contains speech"""
        try:
            return self.vad.is_speech(audio_data.tobytes(), self.sample_rate)
        except:
            return False

class AudioSaver:
    """Handles saving audio to files"""
    @staticmethod
    def save_to_wav(filename: str, audio_data: np.ndarray, sample_rate: int) -> None:
        """Save audio data to WAV file"""
        if audio_data is None:
            print("No audio data to save")
            return
            
        with wave.open(filename, 'wb') as wf:
            wf.setnchannels(1)
            wf.setsampwidth(2)  # 16-bit audio
            wf.setframerate(sample_rate)
            wf.writeframes(audio_data.tobytes())
        print(f"Recording saved to {filename}")

class VoiceRecorder:
    """Main voice recorder class with speech detection and manual stop capability"""
    
    def __init__(self, config: RecorderConfig = RecorderConfig()):
        self.config = config
        self.chunk_size = int(config.sample_rate * config.chunk_duration_ms / 1000)
        self.audio_processor = AudioProcessor(config.sample_rate)
        self.reset_state()
        
    def reset_state(self) -> None:
        """Reset all recording state variables"""
        self.recorded_data: List[np.ndarray] = []
        self.silent_chunks = 0
        self.speech_detected = False
        self.should_stop = False
        
    def stop_recording(self) -> None:
        """Stop the recording when called"""
        self.should_stop = True
        print("\nStopping recording...")
        
    def setup_keyboard_listener(self) -> None:
        """Set up the keyboard listener for manual stop"""
        keyboard.on_press_key(
            self.config.stop_key, 
            lambda _: self.stop_recording()
        )
        
    def cleanup_keyboard_listener(self) -> None:
        """Clean up keyboard listener"""
        keyboard.unhook_all()
        
    def process_audio_chunk(self, indata: np.ndarray) -> bool:
        """Process a chunk of audio data and return True if recording should stop"""
        # Convert audio format
        audio_chunk = self.audio_processor.convert_to_int16(indata)
        
        # Check for speech
        if self.audio_processor.is_speech(audio_chunk):
            self.speech_detected = True
            self.silent_chunks = 0
            self.recorded_data.append(audio_chunk.copy())
        elif self.speech_detected:
            self.silent_chunks += 1
            self.recorded_data.append(audio_chunk.copy())
            
        # Check if we should stop due to silence
        if self.silent_chunks > self.config.silence_threshold:
            print("\nSilence detected, stopping recording...")
            return True
            
        return False
        
    def audio_callback(self, indata: np.ndarray, frames: int, time: any, status: any) -> None:
        """Callback function for audio stream"""
        if status:
            print(status)
            
        if self.should_stop or self.process_audio_chunk(indata):
            raise sd.CallbackStop()
            
    def record(self) -> Optional[np.ndarray]:
        """Record audio until silence is detected or stop key is pressed"""
        print(f"Listening... Speak now (press '{self.config.stop_key}' to stop manually or wait for silence)")
        self.reset_state()
        self.setup_keyboard_listener()
        
        try:
            with sd.InputStream(
                samplerate=self.config.sample_rate,
                channels=1,
                callback=self.audio_callback,
                dtype=np.float32,
                blocksize=self.chunk_size
            ):
                while True:
                    sd.sleep(100)
        except sd.CallbackStop:
            pass
        finally:
            self.cleanup_keyboard_listener()
            
        return np.vstack(self.recorded_data) if self.recorded_data else None



# Create custom configuration if needed
config = RecorderConfig(
    sample_rate=16000,
    chunk_duration_ms=30,
    silence_threshold=30,
    stop_key='esc'
)

# Initialize and use recorder
recorder = VoiceRecorder(config)
audio_data = recorder.record()

# Save recording if we have data
if audio_data is not None:
    saver = AudioSaver()
    saver.save_to_wav("recording.wav", audio_data, config.sample_rate)



: 

In [2]:
import whisper
import os

def transcribe_audio(audio_file_path: str, model_name: str = "base") -> str:
    """
    Transcribe an audio file using OpenAI's Whisper model.
    
    Args:
        audio_file_path (str): Path to the audio file to transcribe
        model_name (str): Whisper model to use (tiny, base, small, medium, large)
    
    Returns:
        str: Transcribed text from the audio file
    
    Raises:
        FileNotFoundError: If the audio file doesn't exist
        ValueError: If the audio file format is not supported
    """
    # Check if file exists
    if not os.path.exists(audio_file_path):
        raise FileNotFoundError(f"Audio file not found: {audio_file_path}")
    
    # Check file extension
    if not audio_file_path.lower().endswith('.mp3'):
        raise ValueError("File must be an MP3 file")
    
    try:
        # Load the Whisper model
        model = whisper.load_model(model_name)
        
        # Transcribe the audio
        result = model.transcribe(audio_file_path)
        
        # Return the transcribed text
        return result["text"]
    
    except Exception as e:
        raise Exception(f"Error during transcription: {str(e)}")

# Example usage:
if __name__ == "__main__":
    try:
        audio_path = "path/to/your/audio.mp3"
        transcription = transcribe_audio(audio_path)
        print("Transcription:")
        print(transcription)
    except Exception as e:
        print(f"Error: {str(e)}")


Error: Audio file not found: path/to/your/audio.mp3
